## Rework TXT files to CSV enhanced ones
* Only if you just have the badly arranged txt files
* Once this file is used, you don't need it anymore 

In [1]:
import pandas as pd
import numpy as np
from os import listdir
from os.path import isfile, join
import os

In [2]:
cwd = os.getcwd()
os.chdir(cwd)
print(cwd)

C:\Users\utilisateur\Workspace_lab


In [3]:
path_GW = r"csv\GW"
path_RG_24 = r"csv\RG\precipiation_RG"
path_RG_1 = r"csv\RG\precipitation_1h_RG"

### Méthodes
***
- prepocess_2(src): return src(dataframe) prend un dataframe pour caster la colonne de Date vers le type Date(pandas).
- prepocess_3(src): return src(dataframe) pareil mais prend en compte l'heure.
- generate_dataset(source): return dataset(dataframe) récupère tous les fichiers du dossier source afin d'extraire au forma Dataframe les données et les pré-traiter.


In [4]:
def preprocess_2(src):
    src["Date"] = pd.to_datetime(src["Date"], format='%d/%m/%Y')
    #src = src.set_index("Date") Plus facile à utiliser sans passer par index, mais joue le rôle de l'index
    return src

In [5]:
def preprocess_3(src):
    src['Date'] = src[['Date', 'Heure']].apply(lambda x: ' '.join(x.astype(str)), axis=1)
    src.drop("Heure", axis=1, inplace=True)
    src["Date"] = pd.to_datetime(src["Date"], format='%d/%m/%Y %H:%M:%S')
    #src = src.set_index("Date")
    return src

In [6]:
from ipywidgets import FloatProgress
from IPython.display import display

def generate_dataset(source):
    f = FloatProgress(min=0, max=100, description="Data preprocessing: ", bar_style="info")
    display(f)
    all_files = [f for f in listdir(cwd +"\\"+ source) if isfile(join(cwd +"\\"+ source, f))]
    dataset = {}
    for txt in all_files:
        data_tmp = pd.read_csv(cwd +"\\"+ source + r"\\" + str(txt), sep=";", header = None)
        if len(data_tmp.columns) == 2:
            data_tmp.columns = ["Date", "Valeur"]
            data_tmp = preprocess_2(data_tmp)
        elif len(data_tmp.columns) == 3:
            data_tmp.columns = ["Date", "Heure", "Valeur"]
            data_tmp = preprocess_3(data_tmp)
        data_tmp["Valeur"] = data_tmp["Valeur"].str.replace(",", ".")
        #dataset[str(txt)] = data_tmp.copy()
        dataset[str(txt)] = data_tmp
        f.value += 100/len(all_files)
    print("Load " + str(source) + ": Done")
    return dataset

In [7]:
data_GW = generate_dataset(path_GW)
data_RG_24 = generate_dataset(path_RG_24)
data_RG_1 = generate_dataset(path_RG_1)

FloatProgress(value=0.0, bar_style='info', description='Data preprocessing: ')

Load csv\GW: Done


FloatProgress(value=0.0, bar_style='info', description='Data preprocessing: ')

Load csv\RG\precipiation_RG: Done


FloatProgress(value=0.0, bar_style='info', description='Data preprocessing: ')

Load csv\RG\precipitation_1h_RG: Done


In [10]:
for key, value in data_GW.items():
    value.to_csv("csv_prepro\GW\\" + str(key[:-3])+ "csv", index=False, sep=";")
for key, value in data_RG_24.items():
    value.to_csv("csv_prepro\RG\precipiation_RG\\" + str(key[:-3])+ "csv", index=False, sep=";")
for key, value in data_RG_1.items():
    value.to_csv("csv_prepro\RG\precipitation_1h_RG\\" + str(key[:-3])+ "csv", index=False, sep=";")